In [18]:
PATH1='Data'
PATH2='Data/TTSLinear/'
PATH3='Data/TTSNonLinear/'

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn import linear_model as lm
from sklearn.ensemble import RandomForestRegressor as rfReg
from sklearn.ensemble import GradientBoostingRegressor as gbReg
from sklearn.ensemble import VotingRegressor as voteReg
from sklearn.ensemble import ExtraTreesRegressor as extraReg
from sklearn.ensemble import HistGradientBoostingRegressor as histReg
from sklearn.ensemble import AdaBoostRegressor as adaReg
from sklearn.ensemble import BaggingRegressor as baggingReg
from sklearn.svm import SVR as svReg
from sklearn.neighbors import KNeighborsRegressor as knReg
from sklearn.gaussian_process import GaussianProcessRegressor as gpReg
from sklearn.tree import DecisionTreeRegressor as treeReg
from sklearn.kernel_ridge import KernelRidge as krReg
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score as cvs
from sklearn.model_selection import train_test_split as tts
from xgboost import XGBRegressor as xgbReg
from lightgbm import LGBMRegressor as lgbmReg
from catboost import CatBoostRegressor as catReg
import numpy as np
import datetime
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import os,shutil
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [7]:
data = pd.read_excel(PATH1+'data solubility.xlsx', skiprows=2, header=None, names=['T', 'P', 'Solubility', 'Molecular weight', 'Melting point', 'Density'])

In [8]:
data

,T,P,Solubility,Molecular weight,Melting point,Density
0,308.0,120.0,0.000003,448.39,415.11,768.42
1,308.0,150.0,0.000004,448.39,415.11,816.06
2,308.0,180.0,0.000005,448.39,415.11,848.87
3,308.0,210.0,0.000005,448.39,415.11,874.40
4,308.0,240.0,0.000006,448.39,415.11,895.54
...,...,...,...,...,...,...
1417,338.0,150.0,0.000003,532.56,470.00,557.00
1418,338.0,180.0,0.000005,532.56,470.00,652.00
1419,338.0,210.0,0.000006,532.56,470.00,710.00
1420,338.0,240.0,0.000007,532.56,470.00,751.00


In [9]:
def plotR2Linear(R2, Y, teks, ts):
  fig, ax1=plt.subplots(1,1, figsize=(7,7))

  ax1.scatter(Y[0], Y[1], c='blue', label='Experiment')
  ax1.plot(Y[1], Y[1], c= "red", marker='.', linestyle='-', label='Prediction')
  ax1.set_title('$R^2=$'+R2+' using '+teks, fontsize=12)
  ax1.set_xlabel('Experiment')
  ax1.set_ylabel('Prediction')
  ax1.legend(loc='upper left')
  ax1.tick_params(right=True)
  fig.savefig(PATH2+' '+teks+'_'+str(ts)+'_.png', dpi=900)
  plt.close(fig)

In [20]:
def plotR2NonLinear(R2, Y, teks, ts):
  fig, ax1=plt.subplots(1,1, figsize=(7,7))

  ax1.scatter(Y[0], Y[1], c='blue', label='Experiment')
  ax1.plot(Y[1], Y[1], c= "red", marker='.', linestyle='-', label='Prediction')
  ax1.set_title('$R^2=$'+R2+' using '+teks, fontsize=12)
  ax1.set_xlabel('Experiment')
  ax1.set_ylabel('Prediction')
  ax1.legend(loc='upper left')
  ax1.tick_params(right=True)
  fig.savefig(PATH2+' '+teks+'_'+str(ts)+'_.png', dpi=900)
  plt.close(fig)

In [11]:
def Linear(ts):
  j=1
  hasil={}
  exp={}

  index=['Lars', 'Lasso', 'LassoLars', 'ElasticNet', 'LinearRegression', 'Ridge', 'BayesianRidge', 'HuberRegressor', 'GammaRegressor', 'PoissonRegressor', 'TweedieRegressor', 'PassiveAggressiveRegressor', 'RANSACRegressor']

  error={}
  error['MSE']=[]
  error['R2']=[]
  error['MAE']=[]
  error['MAPE']=[]

  X=data[['T', 'P', 'Molecular weight', 'Melting point', 'Density']]
  Y=data['Solubility']

  x_train, x_test, y_train, y_test = tts(X,Y,test_size=ts, shuffle=True)
  scaler = StandardScaler()
  scaler.fit(x_train)
  x_train = scaler.transform(x_train)
  x_test = scaler.transform(x_test)

  prediction={}
  regLars = lm.Lars()
  regLars.fit(x_train,y_train)
  yPredictLars=regLars.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictLars)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictLars)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictLars)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictLars)))
  comparison=[y_test, yPredictLars]
  prediction['Y']=y_test.to_list()
  prediction['yPredictLars']=yPredictLars.tolist()
  df1=pd.DataFrame(prediction, columns=['Y','yPredictLars'])
  plotR2Linear(error['R2'][0], comparison, index[0], ts)

  regLasso = lm.Lasso()
  regLasso.fit(x_train,y_train)
  yPredictLasso=regLasso.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictLasso)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictLasso)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictLasso)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictLasso)))
  comparison=[y_test, yPredictLasso]
  prediction['Y']=y_test.to_list()
  prediction['yPredictLasso']=yPredictLasso.tolist()
  df2=pd.DataFrame(prediction, columns=['Y','yPredictLasso'])
  plotR2Linear(error['R2'][1], comparison, index[1], ts)

  regLLars = lm.LassoLars()
  regLLars.fit(x_train,y_train)
  yPredictLLars=regLLars.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictLLars)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictLLars)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictLLars)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictLLars)))
  comparison=[y_test, yPredictLLars]
  prediction['Y']=y_test.to_list()
  prediction['yPredictLLars']=yPredictLLars.tolist()
  df3=pd.DataFrame(prediction, columns=['Y','yPredictLLars'])
  plotR2Linear(error['R2'][2], comparison, index[2], ts)

  regElastic = lm.ElasticNet()
  regElastic.fit(x_train,y_train)
  yPredictElastic=regElastic.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictElastic)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictElastic)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictElastic)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictElastic)))
  comparison=[y_test, yPredictElastic]
  prediction['Y']=y_test.to_list()
  prediction['yPredictElastic']=yPredictElastic.tolist()
  df4=pd.DataFrame(prediction, columns=['Y','yPredictElastic'])
  plotR2Linear(error['R2'][3], comparison, index[3], ts)

  regr = lm.LinearRegression()
  regr.fit(x_train,y_train)
  yPredict=regr.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredict)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredict)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredict)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredict)))
  comparison=[y_test, yPredict]
  prediction['Y']=y_test.to_list()
  prediction['yPredictLR']=yPredict.tolist()
  df5=pd.DataFrame(prediction, columns=['Y','yPredictLR'])
  plotR2Linear(error['R2'][4], comparison, index[4], ts)

  prediction={}
  regrRR=lm.Ridge()
  regrRR.fit(x_train,y_train)
  yPredictRR=regrRR.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictRR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictRR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictRR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictRR)))
  comparison=[y_test, yPredictRR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictRR']=yPredictRR.tolist()
  df6=pd.DataFrame(prediction, columns=['Y','yPredictRR'])
  plotR2Linear(error['R2'][5], comparison, index[5], ts)

  prediction={}
  regrBRidge=lm.BayesianRidge()
  regrBRidge.fit(x_train,y_train)
  yPredictBRidge=regrBRidge.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictBRidge)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictBRidge)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictBRidge)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictBRidge)))
  comparison=[y_test, yPredictBRidge]
  prediction['Y']=y_test.to_list()
  prediction['yPredictBRidge']=yPredictBRidge.tolist()
  df7=pd.DataFrame(prediction, columns=['Y','yPredictBRidge'])
  plotR2Linear(error['R2'][6], comparison, index[6], ts)

  prediction={}
  regrHR=lm.HuberRegressor()
  regrHR.fit(x_train,y_train)
  yPredictHR=regrHR.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictHR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictHR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictHR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictHR)))
  comparison=[y_test, yPredictHR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictHR']=yPredictHR.tolist()
  df8=pd.DataFrame(prediction, columns=['Y','yPredictHR'])
  plotR2Linear(error['R2'][7], comparison, index[7], ts)

  prediction={}
  gammaReg=lm.GammaRegressor()
  gammaReg.fit(x_train,y_train)
  yPredictGamma=gammaReg.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictGamma)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictGamma)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictGamma)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictGamma)))
  comparison=[y_test, yPredictGamma]
  prediction['Y']=y_test.to_list()
  prediction['yPredictGamma']=yPredictGamma.tolist()
  df9=pd.DataFrame(prediction, columns=['Y','yPredictGamma'])
  plotR2Linear(error['R2'][8], comparison, index[8], ts)

  prediction={}
  regrPoisson=lm.PoissonRegressor()
  regrPoisson.fit(x_train,y_train)
  yPredictPoisson=regrPoisson.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictPoisson)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictPoisson)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictPoisson)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictPoisson)))
  comparison=[y_test, yPredictPoisson]
  prediction['Y']=y_test.to_list()
  prediction['yPredictPoisson']=yPredictPoisson.tolist()
  df10=pd.DataFrame(prediction, columns=['Y','yPredictPoisson'])
  plotR2Linear(error['R2'][9], comparison, index[9], ts)

  prediction={}
  regrTweedie=lm.TweedieRegressor()
  regrTweedie.fit(x_train,y_train)
  yPredictTweedie=regrTweedie.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictTweedie)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictTweedie)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictTweedie)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictTweedie)))
  comparison=[y_test, yPredictTweedie]
  prediction['Y']=y_test.to_list()
  prediction['yPredictTweedie']=yPredictTweedie.tolist()
  df11=pd.DataFrame(prediction, columns=['Y','yPredictTweedie'])
  plotR2Linear(error['R2'][10], comparison, index[10], ts)

  prediction={}
  regrPAggresive=lm.PassiveAggressiveRegressor()
  regrPAggresive.fit(x_train,y_train)
  yPredictPAggresive=regrPAggresive.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictPAggresive)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictPAggresive)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictPAggresive)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictPAggresive)))
  comparison=[y_test, yPredictPAggresive]
  prediction['Y']=y_test.to_list()
  prediction['yPredictPAggresive']=yPredictPAggresive.tolist()
  df12=pd.DataFrame(prediction, columns=['Y','yPredictPAggresive'])
  plotR2Linear(error['R2'][11], comparison, index[11], ts)

  prediction={}
  ransacReg=lm.RANSACRegressor()
  ransacReg.fit(x_train,y_train)
  yPredictRansac=ransacReg.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictRansac)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictRansac)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictRansac)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictRansac)))
  comparison=[y_test, yPredictRansac]
  prediction['Y']=y_test.to_list()
  prediction['yPredictRansac']=yPredictRansac.tolist()
  df13=pd.DataFrame(prediction, columns=['Y','yPredictRansac'])
  plotR2Linear(error['R2'][12], comparison, index[12], ts)

  with pd.ExcelWriter(PATH2+'_prediction_'+str(ts)+'_.xlsx') as writer:
    df1.to_excel(writer, sheet_name='Lars Regression', index=False)
    df2.to_excel(writer, sheet_name='Lasso Regression', index=False)
    df3.to_excel(writer, sheet_name='LassoLars Regression', index=False)
    df4.to_excel(writer, sheet_name='ElasticNet Regression', index=False)
    df5.to_excel(writer, sheet_name='Linear Regression', index=False)
    df6.to_excel(writer, sheet_name='Ridge Regression', index=False)
    df7.to_excel(writer, sheet_name='Bayesian Ridge Regression', index=False)
    df8.to_excel(writer, sheet_name='Huber Regression', index=False)
    df9.to_excel(writer, sheet_name='Gamma Regression', index=False)
    df10.to_excel(writer, sheet_name='Poisson Regression', index=False)
    df11.to_excel(writer, sheet_name='Tweedie Regression', index=False)
    df12.to_excel(writer, sheet_name='Passive Aggressive Regression', index=False)
    df13.to_excel(writer, sheet_name='RANSAC Regression', index=False)

  df=pd.DataFrame(error, index=index, columns=['MSE','R2','MAE','MAPE'])
  df.to_excel(PATH2+'_error_'+str(ts)+'_.xlsx')

In [19]:
def NonLinear(ts):
  j=1
  hasil={}
  exp={}

  index=['ExtraTreesRegressor', 'RandomForestRegressor', 'GradientBoostingRegressor', 'HistGradientBoostingRegressor', 'AdaBoostRegressor', 'BaggingRegressor', 'XGBRegressor', 'LGBMRegressor', 'CatBoostRegressor', 'KNeighborsRegressor', 'GaussianProcessRegressor', 'DecisionTreeRegressor','KernelRidgeRegressor', 'SVR', 'VotingRegressor']

  error={}
  error['MSE']=[]
  error['R2']=[]
  error['MAE']=[]
  error['MAPE']=[]

  X=data[['T', 'P', 'Molecular weight', 'Melting point', 'Density']]
  Y=data['Solubility']

  x_train, x_test, y_train, y_test = tts(X,Y,test_size=ts, shuffle=True)
  scaler = StandardScaler()
  scaler.fit(x_train)
  x_train = scaler.transform(x_train)
  x_test = scaler.transform(x_test)

  prediction={}
  regExtra=extraReg()
  regExtra.fit(x_train,y_train)
  yPredictExtra=regExtra.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictExtra)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictExtra)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictExtra)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictExtra)))
  comparison=[y_test, yPredictExtra]
  prediction['Y']=y_test.to_list()
  prediction['yPredictExtra']=yPredictExtra.tolist()
  df1=pd.DataFrame(prediction, columns=['Y','yPredictExtra'])
  plotR2NonLinear(error['R2'][0], comparison, index[0], ts)

  prediction={}
  regRFR=rfReg()
  regRFR.fit(x_train,y_train)
  yPredictRFR=regRFR.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictRFR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictRFR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictRFR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictRFR)))
  comparison=[y_test, yPredictRFR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictRFR']=yPredictRFR.tolist()
  df2=pd.DataFrame(prediction, columns=['Y','yPredictRFR'])
  plotR2NonLinear(error['R2'][1], comparison, index[1], ts)

  prediction={}
  regGBR=gbReg()
  regGBR.fit(x_train,y_train)
  yPredictGBR=regGBR.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictGBR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictGBR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictGBR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictGBR)))
  comparison=[y_test, yPredictGBR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictGBR']=yPredictGBR.tolist()
  df3=pd.DataFrame(prediction, columns=['Y','yPredictGBR'])
  plotR2NonLinear(error['R2'][2], comparison, index[2], ts)

  prediction={}
  regHist=histReg()
  regHist.fit(x_train,y_train)
  yPredictHist=regHist.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictHist)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictHist)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictHist)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictHist)))
  comparison=[y_test, yPredictHist]
  prediction['Y']=y_test.to_list()
  prediction['yPredictHist']=yPredictHist.tolist()
  df4=pd.DataFrame(prediction, columns=['Y','yPredictHist'])
  plotR2NonLinear(error['R2'][3], comparison, index[3], ts)

  prediction={}
  regAda=adaReg()
  regAda.fit(x_train,y_train)
  yPredictAda=regAda.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictAda)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictAda)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictAda)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictAda)))
  comparison=[y_test, yPredictAda]
  prediction['Y']=y_test.to_list()
  prediction['yPredictAda']=yPredictAda.tolist()
  df5=pd.DataFrame(prediction, columns=['Y','yPredictAda'])
  plotR2NonLinear(error['R2'][4], comparison, index[4], ts)

  prediction={}
  regBagging=baggingReg()
  regBagging.fit(x_train,y_train)
  yPredictBagging=regBagging.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictBagging)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictBagging)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictBagging)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictBagging)))
  comparison=[y_test, yPredictBagging]
  prediction['Y']=y_test.to_list()
  prediction['yPredictBagging']=yPredictBagging.tolist()
  df6=pd.DataFrame(prediction, columns=['Y','yPredictBagging'])
  plotR2NonLinear(error['R2'][5], comparison, index[5], ts)

  prediction={}
  regXGB=xgbReg()
  regXGB.fit(x_train,y_train)
  yPredictXGB=regXGB.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictXGB)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictXGB)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictXGB)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictXGB)))
  comparison=[y_test, yPredictXGB]
  prediction['Y']=y_test.to_list()
  prediction['yPredictXGB']=yPredictXGB.tolist()
  df7=pd.DataFrame(prediction, columns=['Y','yPredictXGB'])
  plotR2NonLinear(error['R2'][6], comparison, index[6], ts)

  prediction={}
  regLGBM=lgbmReg()
  regLGBM.fit(x_train,y_train)
  yPredictLGBM=regLGBM.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictLGBM)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictLGBM)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictLGBM)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictLGBM)))
  comparison=[y_test, yPredictLGBM]
  prediction['Y']=y_test.to_list()
  prediction['yPredictLGBM']=yPredictLGBM.tolist()
  df8=pd.DataFrame(prediction, columns=['Y','yPredictLGBM'])
  plotR2NonLinear(error['R2'][7], comparison, index[7], ts)

  prediction={}
  regCat=catReg()
  regCat.fit(x_train,y_train)
  yPredictCat=regCat.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictCat)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictCat)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictCat)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictCat)))
  comparison=[y_test, yPredictCat]
  prediction['Y']=y_test.to_list()
  prediction['yPredictCat']=yPredictCat.tolist()
  df9=pd.DataFrame(prediction, columns=['Y','yPredictCat'])
  plotR2NonLinear(error['R2'][8], comparison, index[8], ts)

  prediction={}
  regrKNR=knReg()
  regrKNR.fit(x_train,y_train)
  yPredictKNR=regrKNR.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictKNR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictKNR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictKNR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictKNR)))
  comparison=[y_test, yPredictKNR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictKNR']=yPredictKNR.tolist()
  df10=pd.DataFrame(prediction, columns=['Y','yPredictKNR'])
  plotR2NonLinear(error['R2'][9], comparison, index[9], ts)

  prediction={}
  regrGPR=gpReg()
  regrGPR.fit(x_train,y_train)
  yPredictGPR=regrGPR.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictGPR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictGPR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictGPR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictGPR)))
  comparison=[y_test, yPredictGPR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictGPR']=yPredictGPR.tolist()
  df11=pd.DataFrame(prediction, columns=['Y','yPredictGPR'])
  plotR2NonLinear(error['R2'][10], comparison, index[10], ts)

  prediction={}
  regrTree=treeReg()
  regrTree.fit(x_train,y_train)
  yPredictTree=regrTree.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictTree)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictTree)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictTree)))
  error['MAPE'].append('{:.3e}'.format(mae(y_test,yPredictTree)))
  comparison=[y_test, yPredictTree]
  prediction['Y']=y_test.to_list()
  prediction['yPredictTree']=yPredictTree.tolist()
  df12=pd.DataFrame(prediction, columns=['Y','yPredictTree'])
  plotR2NonLinear(error['R2'][11], comparison, index[11], ts)

  prediction={}
  regrKernel=krReg()
  regrKernel.fit(x_train,y_train)
  yPredictKR=regrKernel.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictKR)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictKR)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictKR)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictKR)))
  comparison=[y_test, yPredictKR]
  prediction['Y']=y_test.to_list()
  prediction['yPredictKR']=yPredictKR.tolist()
  df13=pd.DataFrame(prediction, columns=['Y','yPredictKR'])
  plotR2NonLinear(error['R2'][12], comparison, index[12], ts)

  prediction={}
  regSV=svReg()
  regSV.fit(x_train,y_train)
  yPredictSV=regSV.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictSV)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictSV)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictSV)))
  error['MAPE'].append('{:.3e}'.format(mae(y_test,yPredictSV)))
  comparison=[y_test, yPredictSV]
  prediction['Y']=y_test.to_list()
  prediction['yPredictSV']=yPredictSV.tolist()
  df14=pd.DataFrame(prediction, columns=['Y','yPredictSV'])
  plotR2NonLinear(error['R2'][13], comparison, index[13], ts)

  prediction={}
  regVote=voteReg(estimators=[('gb', regGBR), ('rf', regRFR), ('xgb', regXGB)])
  regVote.fit(x_train,y_train)
  yPredictVote=regVote.predict(x_test)
  error['MSE'].append('{:.3e}'.format(mse(y_test,yPredictVote)))
  error['R2'].append('{:.3e}'.format(r2(y_test,yPredictVote)))
  error['MAE'].append('{:.3e}'.format(mae(y_test,yPredictVote)))
  error['MAPE'].append('{:.3e}'.format(mape(y_test,yPredictVote)))
  comparison=[y_test, yPredictVote]
  prediction['Y']=y_test.to_list()
  prediction['yPredictVote']=yPredictVote.tolist()
  df15=pd.DataFrame(prediction, columns=['Y','yPredictVote'])
  plotR2NonLinear(error['R2'][14], comparison, index[14], ts)

  with pd.ExcelWriter(PATH2+'_prediction_'+str(ts)+'_.xlsx') as writer:
    df1.to_excel(writer, sheet_name='ExtraTreesRegressor', index=False)
    df2.to_excel(writer, sheet_name='RandomForestRegressor', index=False)
    df3.to_excel(writer, sheet_name='GradientBoostingRegressor', index=False)
    df4.to_excel(writer, sheet_name='HistGradientBoostingRegressor', index=False)
    df5.to_excel(writer, sheet_name='AdaBoostRegressor', index=False)
    df6.to_excel(writer, sheet_name='BaggingRegressor', index=False)
    df7.to_excel(writer, sheet_name='XGBRegressor', index=False)
    df8.to_excel(writer, sheet_name='LGBMRegressor', index=False)
    df9.to_excel(writer, sheet_name='CatBoostRegressor', index=False)
    df10.to_excel(writer, sheet_name='KNeighborsRegressor', index=False)
    df11.to_excel(writer, sheet_name='GaussianProcessRegressor', index=False)
    df12.to_excel(writer, sheet_name='DecisionTreeRegressor', index=False)
    df13.to_excel(writer, sheet_name='KernelRidgeRegressor', index=False)
    df14.to_excel(writer, sheet_name='SVR', index=False)
    df15.to_excel(writer, sheet_name='VotingRegressor', index=False)

  df=pd.DataFrame(error, index=index, columns=['MSE','R2','MAE','MAPE'])
  df.to_excel(PATH2+'_error_'+str(ts)+'_.xlsx')

In [ ]:
#Define your train-test split scheme in variable skema
skema=[0.1, 0.2, 0.25, 0.4]
for i in skema:
  Linear(i)
  NonLinear(i)

0.1


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000363 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 1279, number of used features: 5
[LightGBM] [Info] Start training from score 0.000269
Learning rate set to 0.042566
0:	learn: 0.0010111	total: 11.2ms	remaining: 11.2s
1:	learn: 0.0009831	total: 12.4ms	remaining: 6.17s
2:	learn: 0.0009528	total: 13.3ms	remaining: 4.43s
3:	learn: 0.0009241	total: 14.3ms	remaining: 3.56s
4:	learn: 0.0008990	total: 15.3ms	remaining: 3.04s
5:	learn: 0.0008719	total: 16.3ms	remaining: 2.7s
6:	learn: 0.0008496	total: 17.2ms	remaining: 2.44s
7:	learn: 0.0008273	total: 19.1ms	remaining: 2.36s
8:	learn: 0.0008025	total: 20.4ms	remaining: 2.24s
9:	learn: 0.0007827	total: 21.8ms	remaining: 2.16s
10:	learn: 0.0007612	total: 23ms	remaining: 2.07s
11:	learn: 0.0007436	total: 24.2ms	remaining: 1.99s
12:	learn: 0.0007235